## OpenAPI Tools with Foundry Agent Service  

![Lab Flow](./Assets/lab_flow.png)

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity jsonref

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import (
    PromptAgentDefinition,
)
import jsonref

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")



print("Finished loading environment variables.")
print(f"Foundry Project Endpoint: {foundry_project_endpoint[:50]+'...'}")
print(f"Model Deployment Name: {model_deployment_name}")


### Setting Up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Initializing the OpenAPI Tool

In [ ]:
with open("./weather_openapi.json", "r") as f:
        openapi_weather = jsonref.loads(f.read())

# Initialize agent OpenApi tool using the read in OpenAPI spec
weather_tool = {
        "type": "openapi",
        "openapi":{
            "name": "weather",
            "spec": openapi_weather,
            "auth": {
                "type": "anonymous"
            },
        }
}

### Setting Up Our Agent with OpenAPI Tool Spec

In [ ]:
agent_name = "weather-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful Weather Agent that provides weather information using the provided OpenAPI Tool",
        tools=[weather_tool]
    ),
)

# printing the agent id
print(f"Agent created (id: {agent.id}, name: {agent.name}, version: {agent.version})")

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# creating an openai client first
openai_client = project_client.get_openai_client()

# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

In [ ]:
user_input = "What's the weather like in New York City today?"

### Chat with The Agent

In [ ]:
response = openai_client.responses.create(
            conversation=conversation.id,
            extra_body = {
                "agent": {
                    "name": agent_name,
                    "type": "agent_reference"
                }
            },
            input = user_input
        )

print("Who is making the API call and with what ID?")
print()
print(f"Weather Agent: {response.output_text}")